In [1]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [3]:
#pip install tensorflow-gpu==2.0.0

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jul  5 04:17:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import os
os.chdir('/drive/My Drive')

In [8]:
import warnings

In [9]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [10]:
IMAGE_SIZE = [299, 299]

In [11]:
inception = InceptionV3(input_shape =IMAGE_SIZE + [3], weights="imagenet", include_top=False)

87916544/87910968 [==============================] - 2s 0us/step


In [12]:
# don't train existing weights
for layer in inception.layers:
  layer.trainable = False

In [13]:
# useful for getting number of classes
folders = glob('/drive/My Drive/IMAGE_RECOGNITION/MASK/train/*')
print(len(folders))

2


In [14]:
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=inception.input, outputs=prediction)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [15]:
from keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [16]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [17]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [18]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('/drive/My Drive/IMAGE_RECOGNITION/MASK/train',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1376 images belonging to 2 classes.


In [19]:
test_set = test_datagen.flow_from_directory('/drive/My Drive/IMAGE_RECOGNITION/MASK/train',
                                                 target_size = (299, 299),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1376 images belonging to 2 classes.


In [21]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

#num_epochs = 1000
#num_batch_size = 32

checkpoint = ModelCheckpoint(filepath='/drive/My Drive/IMAGE_RECOGNITION/MASK/my_model.h5', 
                               verbose=1, save_best_only=True)

callbacks = [checkpoint, lr_reducer]

start = datetime.now()

model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=300,
  validation_steps=15,
    callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/30
300/300 [==============================] - 349s 1s/step - loss: 0.4227 - accuracy: 0.9529 - val_loss: 1.2685 - val_accuracy: 0.9271

Epoch 00001: val_loss improved from inf to 1.26850, saving model to /drive/My Drive/IMAGE_RECOGNITION/MASK/my_model.h5
Epoch 2/30
300/300 [==============================] - 195s 650ms/step - loss: 0.7430 - accuracy: 0.9595 - val_loss: 0.8660 - val_accuracy: 0.9667

Epoch 00002: val_loss improved from 1.26850 to 0.86605, saving model to /drive/My Drive/IMAGE_RECOGNITION/MASK/my_model.h5
Epoch 3/30
300/300 [==============================] - 196s 653ms/step - loss: 0.4687 - accuracy: 0.9742 - val_loss: 0.0000e+00 - val_accuracy: 0.9896

Epoch 00003: val_loss improved from 0.86605 to 0.00000, saving model to /drive/My Drive/IMAGE_RECOGNITION/MASK/my_model.h5
Epoch 4/30
300/300 [==============================] - 196s 654ms/step - loss: 0.4667 - accuracy: 0.9767 - val_loss: 7.4132e-07 - val_accuracy: 0.9833

Epoch 00004: val_loss did not improve from

In [22]:
model.evaluate(train_set)

43/43 [==============================] - 29s 670ms/step


[0.0, 0.9941860437393188]

In [23]:
model.evaluate(test_set)

43/43 [==============================] - 29s 673ms/step


[0.0, 0.992732584476471]

In [25]:

from tensorflow import keras
model = keras.models.load_model('/drive/My Drive/IMAGE_RECOGNITION/MASK/my_model.h5',)

In [36]:
# Part 3 - Making new predictions
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/drive/My Drive/IMAGE_RECOGNITION/MASK/train/with mask/0-with-mask.jpg', target_size = (299,299))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

train_set.class_indices
if result[0][0] == 0:
  prediction = 'with_mask'
  print(prediction)

else:
  prediction = 'without_mask'
  print(prediction)

with_mask
